Source: <https://cogo-sys-data.s3.amazonaws.com/index.html> 

I read in a locally stored csv file as df and display its properties and header to get a data overview.

In [7]:
import pandas as pd
import os, sys

#reads in csv sources as df
path = "CoGo_Bikerental_colorado_US/original_data/"
filelist = os.listdir(path)
print(filelist)

#creates counter for entries before and after cleaning
count_original = 0
count_clean = 0

for f in filelist:
    source = path + f
    df = pd.read_csv(source)
    
    #giving a first insight into data and count original entries
    #df.info()
    #display(df.head())
    count_original += len(df)

    #converts timestamps from str format into datetime format (required for calculation of trip duration)
    df['started_at']= pd.to_datetime(df['started_at'])
    df['ended_at']= pd.to_datetime(df['ended_at'])

    #calculates trip duration and adds value to a new attribute "trip duration" 
    df["trip_duration"] = (df["ended_at"] - df["started_at"])

    #filters out trips < 10 min that end at the same coordinates as they start AND trips > 10 h
    maxduration = pd.Timedelta(hours=10)
    minduration = pd.Timedelta(minutes=10)

    df =df[~((df['trip_duration'] < minduration) & (df['start_lat'] == df['end_lat']) & (df['start_lng'] == df['end_lng']))]
    df = df[df['trip_duration'] < maxduration]

    #check on filtered out data
    #rest = df[((df['trip_duration'] < minduration) & (df['start_lat'] == df['end_lat']) & (df['start_lng'] == df['end_lng']))]
    #rest = df[df['trip_duration'] >= maxduration]
    #display(rest.head())

    #for checking the modifications and counting total entries of cleaned data
    #df.info()
    #display(df.head())
    count_clean += len(df)

    #storing cleaned data
    output = "CoGo_Bikerental_colorado_US/cleaned_data/"
    f = "cleaned" + f
    storage = output + f 
    df.to_csv(storage)

#calculates total sum of removed entries over all datasets
removed = count_original - count_clean
share_removed = round((removed/count_original), 3)

print(f"From a total number of  {count_original} entries, {removed} entries ({share_removed} %) have been removed.")

['202007-cogo-tripdata.csv', '202012-cogo-tripdata.csv', '202107-cogo-tripdata.csv', '202112-cogo-tripdata.csv', '202207-cogo-tripdata.csv', '202212-cogo-tripdata.csv', '202307-cogo-tripdata.csv', '202312-cogo-tripdata.csv']
From a total number of  37203 entries, 1399 entries (0.038 %) have been removed.
